In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from epiweeks import Week
from datetime import date, datetime
from os.path import dirname, join, isfile, isdir

from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_igbe
from bulletin.commom.utils import isvaliddate

from bulletin.commom.utils import Timer, auto_fit_columns
output = join("output")

In [2]:
municipios = static.municipios.loc[static.municipios['uf']=='PR',['ibge','municipio','populacao']].copy()
municipios['ibge'] = municipios['ibge'].apply(str)

regionais = static.regionais[['ibge','nm_macro','nu_reg','nm_reg']].copy()
regionais['ibge'] = regionais['ibge'].apply(str)
regionais = regionais.rename(columns={'nu_reg':'rs'})

populacao_macro = pd.merge(regionais,municipios,on='ibge',how='inner').groupby('nm_macro')[['populacao']].sum()
populacao_rs = pd.merge(regionais,municipios,on='ibge',how='inner').groupby('rs')[['populacao']].sum()
populacao_mun = municipios

In [3]:
casos = pd.read_excel('input/casos_obitos.xlsx','casos',converters={
        'ibge': normalize_igbe
    }
)

In [4]:
obitos = pd.read_excel('input/casos_obitos.xlsx','obitos',converters={
        'ibge': normalize_igbe
    }
)

In [5]:
casos = pd.merge(left=casos, right=regionais, how='left', on='ibge')
casos = pd.merge(left=casos, right=municipios, how='left', on='ibge')
casos['ano'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).year, axis=1)
casos['mes'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).startdate().month, axis=1)
casos['se'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).week, axis=1)

In [6]:
obitos = pd.merge(left=obitos, right=regionais, how='left', on='ibge')
obitos = pd.merge(left=obitos, right=municipios, how='left', on='ibge')
obitos['ano'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).year, axis=1)
obitos['mes'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).startdate().month, axis=1)
obitos['se'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).week, axis=1)

In [7]:
casos_se = casos.groupby(['ano','mes','se','nm_macro','rs','nm_reg','ibge','municipio','populacao'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})

casos_se = casos_se.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
casos_se = casos_se.xs('casos', axis=1, level= 3)
casos_se

ano                                                            2020           \
mes                                                              3             
se                                                               11 12 13 14   
nm_macro rs nm_reg     ibge   municipio              populacao                 
LESTE    1   PARANAGUA 410120 Antonina               18949        0  0  0  0   
                       410950 Guaraqueçaba           7594         0  0  0  0   
                       410960 Guaratuba              37527        0  0  0  0   
                       411570 Matinhos               35219        0  0  0  2   
                       411620 Morretes               16446        0  0  0  0   
...                                                             ... .. .. ..   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627         0  0  0  0   
                       412575 São Pedro do Iguaçu    5820         0  0  0  0   
                       412740 Terra Roxa             17522        0  0  0  0   
                       412770 Toledo                 142645       0  0  1  3   
                       412795 Tupãssi                8109         0  0  0  0   

ano                                                                            \
mes                                                            4           5    
se                                                             15 16 17 18 19   
nm_macro rs nm_reg     ibge   municipio              populacao                  
LESTE    1   PARANAGUA 410120 Antonina               18949      0  0  0  0  0   
                       410950 Guaraqueçaba           7594       0  0  0  0  1   
                       410960 Guaratuba              37527      0  0  2  1  2   
                       411570 Matinhos               35219      0  0  0  0  0   
                       411620 Morretes               16446      0  1  0  1  0   
...                                                            .. .. .. .. ..   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627       0  0  0  0  0   
                       412575 São Pedro do Iguaçu    5820       0  0  0  0  1   
                       412740 Terra Roxa             17522      0  0  0  0  0   
                       412770 Toledo                 142645     6  1  0  5  0   
                       412795 Tupãssi                8109       0  1  0  0  0   

ano                                                                ...       \
mes                                                                ...   12   
se                                                             20  ...   52   
nm_macro rs nm_reg     ibge   municipio              populacao     ...        
LESTE    1   PARANAGUA 410120 Antonina               18949      0  ...   38   
                       410950 Guaraqueçaba           7594       0  ...    1   
                       410960 Guaratuba              37527      1  ...   48   
                       411570 Matinhos               35219      0  ...   65   
                       411620 Morretes               16446      0  ...   34   
...                                                            ..  ...  ...   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627       0  ...    1   
                       412575 São Pedro do Iguaçu    5820       0  ...   14   
                       412740 Terra Roxa             17522      0  ...   20   
                       412770 Toledo                 142645     3  ...  422   
                       412795 Tupãssi                8109       0  ...   11   

ano                                                                 2021       \
mes                                                                   1         
se                                                               53   1    2    
nm_macro rs nm_reg     ibge   municipio              populacao                  
LESTE    1   PARANAGUA 410120 Antonina               18949       43   62 

In [8]:
obitos_se = obitos.groupby(['ano','mes','se','nm_macro','rs','nm_reg','ibge','municipio','populacao'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})

obitos_se = obitos_se.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
obitos_se = obitos_se.xs('obitos', axis=1, level= 3)
obitos_se

ano                                                            2020           \
mes                                                              3     4       
se                                                               13 14 15 16   
nm_macro rs nm_reg     ibge   municipio              populacao                 
LESTE    1   PARANAGUA 410120 Antonina               18949        0  0  0  0   
                       410950 Guaraqueçaba           7594         0  0  0  0   
                       410960 Guaratuba              37527        0  0  0  0   
                       411570 Matinhos               35219        0  0  0  0   
                       411620 Morretes               16446        0  0  0  0   
...                                                             ... .. .. ..   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627         0  0  0  0   
                       412575 São Pedro do Iguaçu    5820         0  0  0  0   
                       412740 Terra Roxa             17522        0  0  0  0   
                       412770 Toledo                 142645       0  0  0  0   
                       412795 Tupãssi                8109         0  0  0  0   

ano                                                                            \
mes                                                                  5          
se                                                             17 18 19 20 21   
nm_macro rs nm_reg     ibge   municipio              populacao                  
LESTE    1   PARANAGUA 410120 Antonina               18949      0  0  0  0  0   
                       410950 Guaraqueçaba           7594       0  0  0  1  0   
                       410960 Guaratuba              37527      0  0  0  0  0   
                       411570 Matinhos               35219      0  0  0  0  0   
                       411620 Morretes               16446      0  0  0  0  0   
...                                                            .. .. .. .. ..   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627       0  0  0  0  0   
                       412575 São Pedro do Iguaçu    5820       0  0  0  0  0   
                       412740 Terra Roxa             17522      0  0  0  0  0   
                       412770 Toledo                 142645     0  0  0  0  0   
                       412795 Tupãssi                8109       0  0  0  0  0   

ano                                                                ...        \
mes                                                                ... 12      
se                                                             22  ... 52 53   
nm_macro rs nm_reg     ibge   municipio              populacao     ...         
LESTE    1   PARANAGUA 410120 Antonina               18949      0  ...  2  0   
                       410950 Guaraqueçaba           7594       0  ...  0  0   
                       410960 Guaratuba              37527      1  ...  2  2   
                       411570 Matinhos               35219      0  ...  1  1   
                       411620 Morretes               16446      0  ...  1  2   
...                                                            ..  ... .. ..   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627       0  ...  0  0   
                       412575 São Pedro do Iguaçu    5820       0  ...  0  0   
                       412740 Terra Roxa             17522      0  ...  0  0   
                       412770 Toledo                 142645     0  ...  3  8   
                       412795 Tupãssi                8109       0  ...  0  2   

ano                                                            2021           \
mes                                                              1             
se                                                               1  2  3  4    
nm_macro rs nm_reg     ibge   municipio              populacao                 
LESTE    1   PARANAGUA 410120 Antonina               18949    

In [9]:
casos_se_aux = casos_se.copy()
# casos_se_aux['total_geral'] = casos_se.sum(axis=1)

# for ano in casos_se.columns.levels[0]:
#     for mes in casos_se.columns.levels[1]:
#         try:
#             casos_se_aux[(ano,mes,'total')] = casos_se[(ano,mes)].sum(axis=1)
#         except:
#             pass

casos_se_aux = casos_se_aux.sort_index(axis=1)

In [10]:
obitos_se_aux = obitos_se.copy()
# obitos_se_aux['total_geral'] = obitos_se.sum(axis=1)

# for ano in obitos_se.columns.levels[0]:
#     for mes in obitos_se.columns.levels[1]:
#         try:
#             obitos_se_aux[(ano,mes,'total')] = obitos_se[(ano,mes)].sum(axis=1)
#         except:
#             pass

obitos_se_aux = obitos_se_aux.sort_index(axis=1)

In [11]:
aux = pd.DataFrame(index=casos_se_aux.index, columns=casos_se_aux.columns, data=0)
aux = pd.merge(aux,obitos_se_aux,how='left',left_index=True,right_index=True, suffixes=['_empty','']).fillna(0).astype(int)
aux = aux.drop(columns=list(filter(lambda x: x, [x if '_empty' in x else None for x in aux.columns.levels[0]])))

In [12]:
aux['2020',3,11] = 0
aux['2020',3,12] = 0
aux = aux.sort_index(axis=1)

In [13]:
casos_se = casos_se_aux
obitos_se = aux



writer = pd.ExcelWriter(join(output,"casos_obitos_se.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos_se.to_excel(writer,sheet_name='casos_se')
obitos_se.to_excel(writer,sheet_name='obitos_se')

worksheet = writer.sheets[f"casos_se"]
auto_fit_columns(worksheet,casos_se)

worksheet = writer.sheets[f"obitos_se"]
auto_fit_columns(worksheet,obitos_se)

writer.save()
writer.close()